# Intro to Numpy

**Attribution note**: This notebook was created by David Dotson and Oliver Beckstein and is made available under a CC-BY 4.0 license. Some of the material was inspired by lessons developed by [Software Carpentry](http://software-carpentry.org/) ([Programming with Python](http://swcarpentry.github.io/python-novice-inflammation/)), as well as material previously created by Oliver Beckstein ([Python and Numpy for SimBioNano PHY 598](http://becksteinlab.physics.asu.edu/pages/courses/2013/SimBioNano/04/PythonAndNumpy/p04_class.html)).

When it comes to doing numerical work, Python by itself is rather slow. By slow we mean compared to languages like C and Fortran, which benefit from being **compiled** languages in which a program is preprocessed into machine code by a compiler. Python by contrast is an **interpreted** language, in which each line in a program is fed to the Python interpreter in sequence, then executed. The flexiblity and ease of use that come with Python come at the cost of pure performance.

However, though Python code itself may be slow, Python can be used to run code that is written in a compiled language and already compiled. We will use a library (a.k.a., a Python *module*) that does exactly this underneath the hood to get fast performance for numerical operations on arrays.

In [ ]:
import numpy

Importing a module is like taking a piece of equipment out of a storage locker and setting it up on a lab bench. Importing the name `numpy` makes all the functions and classes (object types) available to us. The core data structure that `numpy` provides is known as the `numpy` array:

In [ ]:
somenums = numpy.array([1, 2, 3, 4])
print(somenums)

A numpy array looks superfically similar to a `list`, which is a builtin to Python. They are fundamentally different, however, in how they both work and how they exist in memory. `numpy` arrays don't store references to other objects, but instead point to contiguous blocks of memory in which each element is of exactly the same data type. For instance, we just made an array of 64 bit integers:

In [ ]:
somenums.dtype

In [ ]:
# this will give an array with a string dtype
numpy.array(['Han Solo', "Kylo Ren", 7])

In [ ]:
# this will give an array of 64 bit floats
floats = numpy.array([63.3, -5.0, 1])
print(floats)

In [ ]:
floats.dtype

Also, because arrays are not a collection of objects but are a single object of identically sized pieces of data, they cannot be resized. To add elements to an array, one must create a new array.

In [ ]:
# this will create a new array with repeated elements
numpy.hstack([somenums, somenums])

## Array methods (or, arrays are objects)

`numpy` arrays are built for numerical operations, and doing them quickly. Since like everything in Python these are *objects*, they include built-in methods such as:

In [ ]:
somenums.mean()

In [ ]:
somenums.std()

and a whole plethora of others. You can get a view of what methods and attributes are part of an array's namespace with:

In [ ]:
dir(somenums)

or in the notebook, by typing the name of the array followed by a `.` and the tab key:

In [ ]:
somenums.

Recall that you can also get the documentation for any function or method with a question mark at the end of the name:

In [ ]:
somenums.mean?

## Multidimensionality, indexing, and slicing

`numpy` arrays can be of any dimensionality, not just 1-D. It's common to encounter 2-D arrays, and for illustration we'll look at the position of a particle in three dimensions with time:

In [ ]:
import numpy as np

In [ ]:
def get_position(nframes=10**6):
    """Get array of x, y, and z position of a particle with time.
    
    Parameters
    ----------
    nframes: int
        number of frames; more frames increases the resolution
        of the trajectory, but not its length
        
    Returns
    -------
    position : `nframes` x 3 array
        (x, y, z) position of the particle with time
        
    """
    # generate x, y, z positions
    x = np.cos(np.linspace(0, 20, nframes))
    y = 3 * np.sin(np.linspace(0, 10, nframes))
    z = -2 * np.sin(np.pi * np.linspace(0, 5, nframes))

    # put them all in a single array; this gives
    # an array with 3 rows and nframes columns
    position = np.array([x, y, z])

    # we want the transpose array for now
    return position.transpose()

Let's generate an array `position` to work with:

In [ ]:
position = get_position()

In [ ]:
position.shape

Now say we wanted to examine the position of the particle in the very first frame (row), we could do:

In [ ]:
position[0]

to extract it. Notice that indexing starts at 0, as is the convention in Python.

What about the third frame?

In [ ]:
position[2]

In zero-based terms, this we would call the "first frame" the zeroth frame, and so on. To avoid confusion we'll assume this from now on.

What if we wanted a bunch of frames, but only the 5th through the 72nd? It should have 68 rows:

In [ ]:
position[5:73].shape

Notice the **slicing** notation. Remember, this should be read as

> "Get each row in the array starting from the row at index 5 up to and not including the row at index 73."

We could even coarse-grain by slicing out every fifth row in this range:

In [ ]:
position[5:73:5].shape

Now what if we wanted a specific *element* of the array? Indexing works for this too:

In [ ]:
position[42, 1]

This is the y-position of the 42nd frame. 

NOTE: **The numpy indexing notation differs from Python indexing of nested lists**, which would look like `positions[42][1]`. (Incidentally, this also works for numpy arrays but is less readable and slower.)

In [ ]:
%timeit position[42, 1]

In [ ]:
%timeit position[42][1]

The first index/slice corresponds to the first *axis* of the array, which for a 2-D array corresponds to the rows. The second index/slice would then be the columns. If we had a 3-D array, indexing the first axis would yield 2-D arrays. If we had a 4-D array, indexing the first axis would yield 3-D arrays, and so on.

### Example: Numpy slicing

Obtain an array giving the mean of the x, y positions (separately) from the frame at index 10 to the frame at index 43 as a 1-D array.

We can do this by slicing both the first axis (rows) and the second axis (columns), then using the `mean` method of the resulting array. To only take a mean across the rows (a mean for each column), we must specify the `axis=0` keyword.

In [ ]:
position[10:42, :2].mean(axis=0)

What if we wanted the smaller of the two numbers only?

In [ ]:
position[10:43, :2].mean(axis=0).min()

Since slicing and methods of arrays often yield arrays, you can chain operations in this way. This is what qualifies as a *pythonic* way to work with these objects.

### Fancy and boolean indexing

It's also possible to index arrays with lists of indices to select out; these can be repeated and in any order.

In [ ]:
position[[2, 4, 7, -1, 2]]

We can also use arrays of booleans to get back arrays with items for which `True` was the value in the boolean array used:

In [ ]:
(position[:, :2] > 2).any(axis=1)

We can use this array to get only the rows for which either the x or y position is greater than 2:

In [ ]:
position[(position[:,:2] > 2).any(axis=1)].shape

Boolean arrays are useful for filtering data for rows of interest.

**Technical note**: fancy and boolean indexing like this generally gives back a new array instead of a *view* to the existing one. Slicing, by contrast, always gives a view. This matters when using indexing or slicing to alter the values in an array.

## Array arithmetic 

In [ ]:
position = get_position()

We can do anything with arrays of the same shape that we can do with single numbers.

In [ ]:
((2 * position + position) * position)**.5

is the same (up to rounding errors — see next lecture) as $\sqrt{(2r + r) \cdot r} = \sqrt{3r^2} = r \sqrt{3}$

In [ ]:
position * 3**.5

Note that multiplication between two arrays is **not** the same as matrix multiplcation. Arithmetic operations are element-wise.

In [ ]:
np.array([0, -1, 10]) * np.array([3, 5, -1])

 But there is a method for doing matrix multiplication:

In [ ]:
np.dot(np.array([0, -1, 10]), np.array([3, 5, -1]))

And more linear algebra functions can be found in the `np.linalg` namespace:

In [ ]:
dir(np.linalg)

## Thinking in arrays 

Say we wanted to displace our particle a full 5 units in each of the x, y, and z directions. If you have experience with a language like C, you might be used to writing nested loops like this one to achieve this:

In [ ]:
position = get_position()

In [ ]:
%%time
for i in range(position.shape[0]):
    for j in range(position.shape[1]):
        position[i, j] += 5

Even slower when you try to do it in a "Pythonic" fashion with the (otherwise very good!) `enumerate()` function:

In [ ]:
position = get_position()

In [ ]:
%%time
for i, row in enumerate(position):
    for j, element in enumerate(row):
        position[i, j] += 5     

In [ ]:
position

But one of the main points of `numpy` is performance, so we'd do better to spend as little time in an operation running through the Python interpreter, which is the case in the above loop. Instead we can do:

In [ ]:
position = get_position()

In [ ]:
%%time
position += 5

Speed-up for using array operations instead of `for` loops:

In [ ]:
2.26 / 14.3e-3

On my laptop the difference in speed is about a factor of 150 (you might see speed-ups on the order of 100 to 1000). The larger the array the more pronounced the difference in speed will be, too. The general rule when using `numpy` is to try and put what you're trying to do in terms of operations on whole arrays (or slices of them). **Avoid Python loops unless absolutely necessary.**

### Example: array arithmetic with *broadcasting*

Rescale (multiply) the y-positions by 2 and displace the x and z positions by 3 and -100, respectively.

There are a lot of ways to do this, but the most succinct way is to take advantage of *broadcasting*. That is, doing:

In [ ]:
position = get_position()

In [ ]:
position = position * np.array([1, 2, 1]) + np.array([3, 0, -100])

`numpy` will take the 3-element, 1-D arrays here and apply them to whole columns in `position`. Note that we already took advantage of broadcasting rules in a way, since multiplying an array by a scalar is the same as multiplying the array by an array of equal shape with all elements equal to the scalar.

# Plotting with matplotlib

> The purpose of computing is insight, not numbers.
>
> -- <cite>Richard Hamming</cite>

We want to do more with arrays than calculate single values or look closely at their elements. A core part of doing science is gaining insight into the patterns revealed by data, for which **visualization** is key. We can produce publication-quality plots from `numpy` arrays with `matplotlib`.

In [ ]:
# we want to use the pyplot interface
import matplotlib.pyplot as plt

In [ ]:
# this tells the notebook to render plots inline
%matplotlib inline

In [ ]:
position = get_position()

Let's plot the x position of our particle as a function of frame:

In [ ]:
plt.plot(position[:, 0])

And then perhaps the x and y position on the x- and y-axis, respectively:

In [ ]:
plt.plot(position[:, 0], position[:, 1])

`matplotlib` is said to make easy things easy and hard things possible when it comes to plotting. Let's make a three-panel plot giving the x, y, and z position as a function of frame.

In [ ]:
fig = plt.figure(figsize=(10,3))

# make an axis object for the figure, each with a different position
# on a grid that has 1 row and 3 columns of axes
ax1 = fig.add_subplot(1, 3, 1)
ax2 = fig.add_subplot(1, 3, 2)
ax3 = fig.add_subplot(1, 3, 3)

# plot the x position
ax1.set_ylabel('x position')
ax1.set_xlabel('frame')
ax1.plot(position[:, 0])

# plot the y position
ax2.set_ylabel('y position')
ax1.set_xlabel('frame')
ax2.plot(position[:, 1])

# plot the z position
ax3.set_ylabel('z position')
ax1.set_xlabel('frame')
ax3.plot(position[:, 2])

fig.tight_layout()

There's plenty more we could do with this to make it prettier. For a primer on making visually appealing and informative plots, check out [this notebook](https://github.com/Becksteinlab/making-prettier-plots/blob/master/Making%20better%20plots.ipynb).

### 3D plotting
Matplotlib has [basic 3D capabilities](http://matplotlib.org/1.3.1/mpl_toolkits/mplot3d/tutorial.html); for more sophisticated plots check out [mayavi](http://docs.enthought.com/mayavi/mayavi/).

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x, y, z = position.transpose()
ax.plot(x, y, z)
ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")
ax.set_zlabel(r"$z$")